In [6]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

### Chat Model

In [10]:
# 일반 챗
messages = [
  SystemMessage('당신은 도우미 어시스턴트입니다.'),
  HumanMessage('안녕하세요! 저는 손이라고 합니다.'),
  AIMessage(content='안녕하세요 손님! 어떤 도움이 필요하신가요?'),
  HumanMessage(content='제 이름을 아시나요?')
]

ai_message = model.invoke(messages)
print(f'response : {ai_message}')
print(f'content : {ai_message.content}')

response : content='네, 당신의 이름은 손이라고 하셨습니다. 어떻게 도와드릴까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 60, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CQR9aBmO7iD4Qyrxumk0uEkgv9I8A', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--ba64c480-1fce-4360-8371-6ef1d30ff5e8-0' usage_metadata={'input_tokens': 60, 'output_tokens': 19, 'total_tokens': 79, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content : 네, 당신의 이름은 손이라고 하셨습니다. 어떻게 도와드릴까요?


In [ ]:
# 스트리밍
messages = [
  SystemMessage('당신은 도우미 어시스턴트입니다.'),
  HumanMessage('안녕하세요!'),
]

for chunk in model.stream(messages):
  print(chunk.content, end="", flush=True)

안녕하세요! 어떻게 도와드릴까요?

### Prompt template

```
PromptTemplate
```
문자열 일부를 대체하는 것뿐, 내부에서 LLM을 호출하지 않음

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""다음 요리의 레시피를 생각해 주세요.

요리명: {dish}""")

prompt_value = prompt.invoke({'dish':'카레'})
print(prompt_value.text)

다음 요리의 레시피를 생각해 주세요.

요리명: 카레


```
ChatPromptTemplate
```
채팅 형식 모뎅에 대응 시킴

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
  [
    ('system', '사용자가 입력한 요리의 레시피를 생각해 주세요.'),
    ('human', '{dish}'),
  ]
)

prompt_value = prompt.invoke({'dish':'카레'})
print(prompt_value)

messages=[SystemMessage(content='사용자가 입력한 요리의 레시피를 생각해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='카레', additional_kwargs={}, response_metadata={})]


```
MessagesPlaceholder
```
대화 이력처럼 여러 메세지가 들어가는 플레이스홀더를 두고 싶은 경우가 많을때 사용

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate(
  [
    ('system', '당신은 도우미 어시스턴트입니다.'),
    MessagesPlaceholder('chat_history', optional=True),
    ('human', '{input}'),
  ]
)

prompt_value = prompt.invoke(
  {
    'chat_history': [
      HumanMessage(content='안녕하세요! 저는 손이라고 합니다.'),
      AIMessage('안녕하세요, 손님! 무엇을 도와드릴까요?'),
    ],
    'input': '제 이름을 아시나요?'
  }
)
print(prompt_value)

messages=[SystemMessage(content='당신은 도우미 어시스턴트입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요! 저는 손이라고 합니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 손님! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='제 이름을 아시나요?', additional_kwargs={}, response_metadata={})]


### Output parser

In [ ]:
from pydantic import BaseModel, Field

# 재료 목록과 순서 정의
class Recipe(BaseModel):
  ingredients: list[str] = Field(description="ingredients of the dish")
  steps: list[str] = Field(description="steps to make the dish")

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser

# 클래스를 제공하여 PydanticOutputParser를 생성
output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [ ]:
# 프롬프트에 출력할 출력 형식 설명문 작성
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [36]:
from langchain_core.prompts import ChatPromptTemplate

# format_instructions를 사용한 ChatPromptTemplate 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "사용자가 입력한 요리의 레시피를 생각해 주세요.\n\n"
            "{format_instructions}",
        ),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=format_instructions
)

In [37]:
prompt_value = prompt_with_format_instructions.invoke({"dish": "카레"})
print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: user ===")
print(prompt_value.messages[1].content)

=== role: system ===
사용자가 입력한 요리의 레시피를 생각해 주세요.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
=== role: user ===
카레


In [38]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

ai_message = model.invoke(prompt_value)
print(ai_message.content)

{
  "ingredients": [
    "닭고기 500g",
    "양파 1개",
    "감자 2개",
    "당근 1개",
    "카레 가루 3큰술",
    "식용유 2큰술",
    "소금 약간",
    "후추 약간",
    "물 4컵"
  ],
  "steps": [
    "양파를 잘게 썰고, 감자와 당근은 큐브 모양으로 자릅니다.",
    "냄비에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶습니다.",
    "닭고기를 넣고 겉면이 노릇해질 때까지 볶습니다.",
    "감자와 당근을 추가하고 함께 볶습니다.",
    "카레 가루를 넣고 잘 섞은 후 물을 부어 끓입니다.",
    "끓기 시작하면 중약불로 줄이고 20분 정도 끓입니다.",
    "소금과 후추로 간을 맞춘 후 불을 끄고 5분 정도 둡니다.",
    "밥과 함께 서빙합니다."
  ]
}


In [39]:
recipe = output_parser.invoke(ai_message)
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['닭고기 500g', '양파 1개', '감자 2개', '당근 1개', '카레 가루 3큰술', '식용유 2큰술', '소금 약간', '후추 약간', '물 4컵'] steps=['양파를 잘게 썰고, 감자와 당근은 큐브 모양으로 자릅니다.', '냄비에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶습니다.', '닭고기를 넣고 겉면이 노릇해질 때까지 볶습니다.', '감자와 당근을 추가하고 함께 볶습니다.', '카레 가루를 넣고 잘 섞은 후 물을 부어 끓입니다.', '끓기 시작하면 중약불로 줄이고 20분 정도 끓입니다.', '소금과 후추로 간을 맞춘 후 불을 끄고 5분 정도 둡니다.', '밥과 함께 서빙합니다.']


### StrOutputParser

In [40]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="안녕하세요. 저는 AI 어시스턴트입니다.")
ai_message = output_parser.invoke(ai_message)
print(type(ai_message))
print(ai_message)

<class 'str'>
안녕하세요. 저는 AI 어시스턴트입니다.


### Chain-LangChain Expression Language (LCEL)

단순 LLM에 입출력 뿐만 아니라 처리를 연쇄적으로 하고 싶은 경우가 있음
```
- prompt template을 채우고, 그 결과를 Chat model에 제공한 후, 그 결과를 Python 객체로 변환하고 싶다.
- Zero-shot CoT 프롬프팅으로 단계별로 생각하게하고, 그 결과를 요약하고 싶다.
- LLM의 출력을 얻은 후에, 그 내요이 서비스 정책에 위반되지 않는지 확인하고 싶다.
- LLM의 출력 결과를 바탕으로 SQL을 실행하여 데이터를 분석하고 싶다.
```
이러한 처리의 연쇄를 구현한 것이 LangChain의 'Chain'

```
LCEL은 LangChain에 Chain을 기술하는 방법
```

In [41]:
# prompt와 model 연결
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate(
  [
    ('system','사용자가 입력한 요리의 레시피를 생각해 주세요.'),
    ('human', '{dish}'),
  ]
)
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

In [42]:
chain = prompt | model

In [43]:
ai_message = chain.invoke({'dish':'카레'})
print(ai_message.content)

카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.

### 재료
- 고기 (닭고기, 소고기, 양고기 등) 300g
- 양파 1개
- 감자 1개
- 당근 1개
- 카레 가루 2-3 큰술
- 식용유 2 큰술
- 물 3컵
- 소금, 후추 약간
- 선택 재료: 마늘, 생강, 피망, 버섯 등

### 조리 방법
1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파는 다지고, 감자와 당근은 깍둑썰기 합니다. 선택 재료가 있다면 함께 손질합니다.

2. **양파 볶기**: 큰 냄비에 식용유를 두르고 중불에서 다진 양파를 넣고 투명해질 때까지 볶습니다.

3. **고기 추가**: 양파가 볶아지면 고기를 넣고 겉면이 익을 때까지 볶습니다.

4. **채소 추가**: 감자와 당근을 넣고 함께 볶아줍니다. 선택 재료가 있다면 이때 추가합니다.

5. **물 붓기**: 모든 재료가 잘 섞이면 물을 붓고 끓입니다. 끓기 시작하면 불을 줄이고 중약불에서 15-20분 정도 끓입니다.

6. **카레 가루 추가**: 카레 가루를 넣고 잘 섞은 후, 다시 10분 정도 끓입니다. 필요에 따라 소금과 후추로 간을 맞춥니다.

7. **완성**: 카레가 걸쭉해지면 불을 끄고, 밥과 함께 서빙합니다.

### 팁
- 카레는 시간이 지날수록 맛이 깊어지므로, 하루 정도 숙성시키면 더욱 맛있습니다.
- 다양한 재료를 추가하여 나만의 카레를 만들어 보세요!

맛있게 드세요!


```
StrOutputParser를 체인에 추가
```

In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
output = chain.invoke({'dish':'z카레'})
print(output)

Z카레는 일본식 카레의 한 종류로, 일반적으로 부드러운 카레 소스와 다양한 재료가 어우러진 요리입니다. 아래는 Z카레의 기본 레시피입니다.

### 재료
- 카레 루(일본식 카레 가루) 1팩
- 닭고기 또는 소고기 300g (한입 크기로 자른 것)
- 감자 1개 (깍둑썰기)
- 당근 1개 (깍둑썰기)
- 양파 1개 (채썰기)
- 물 4컵
- 식용유 1큰술
- 소금, 후추 (기호에 따라)

### 조리 방법
1. **재료 준비**: 모든 재료를 손질하여 준비합니다. 고기는 한입 크기로 자르고, 감자와 당근은 깍둑썰기, 양파는 채썰어 둡니다.

2. **고기 볶기**: 큰 냄비에 식용유를 두르고 중불에서 양파를 볶아 투명해질 때까지 볶습니다. 그 후, 고기를 넣고 겉면이 노릇해질 때까지 볶습니다.

3. **채소 추가**: 볶은 고기에 감자와 당근을 추가하고, 1-2분 더 볶습니다.

4. **물 붓기**: 냄비에 물을 붓고 끓입니다. 끓기 시작하면 불을 줄이고, 뚜껑을 덮고 약 15-20분간 끓입니다. 이때 재료가 부드러워질 때까지 조리합니다.

5. **카레 루 추가**: 카레 루를 조금씩 넣으면서 잘 저어줍니다. 카레 루가 완전히 녹을 때까지 저어주고, 다시 끓입니다.

6. **간 맞추기**: 소금과 후추로 간을 맞추고, 원하는 농도가 될 때까지 약한 불에서 5-10분 더 끓입니다.

7. **서빙**: 완성된 Z카레를 밥 위에 얹어 서빙합니다. 기호에 따라 피클이나 샐러드를 곁들여도 좋습니다.

맛있게 드세요!


```
PydanticOutputParser를 사용한 체인
```

In [48]:
# Recipe 클래스 정의
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field

class Recipe(BaseModel):
  ingredients: list[str] = Field(description="ingredients of the dish")
  steps: list[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [ ]:
# 체인으로 연결
from langchain_core.prompts import ChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate(
  [
    ('system', '사용자가 입력한 요리의 레시피를 생각해 주세요.\n\n{format_instructions}'),
    ('human', '{dish}'),
  ]
)

prompt_with_format_instructions = prompt.partial(
  format_instructions=output_parser.get_format_instructions()
)

model = ChatOpenAI(model='gpt-4o-mini', temperature=0).bind(
  response_format={'type': 'json_object'}
)

In [50]:
chain = prompt_with_format_instructions | model | output_parser

recipe = chain.invoke({'dish':'카레'})
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['닭고기 500g', '양파 1개', '감자 2개', '당근 1개', '카레 가루 3큰술', '식용유 2큰술', '소금 약간', '후추 약간', '물 4컵'] steps=['닭고기를 한 입 크기로 자르고, 소금과 후추로 간을 한다.', '양파는 다지고, 감자와 당근은 깍둑썰기로 준비한다.', '팬에 식용유를 두르고 다진 양파를 넣어 볶는다.', '양파가 투명해지면 닭고기를 넣고 볶는다.', '닭고기가 익으면 감자와 당근을 넣고 함께 볶는다.', '카레 가루를 넣고 잘 섞은 후 물을 부어 끓인다.', '중불에서 20분 정도 끓여서 재료가 부드러워지면 완성한다.']
